# How far can we push linear models in LB ? :)

check out these forerunners too

* https://www.kaggle.com/julian3833/jigsaw-incredibly-simple-naive-bayes-0-768 
* https://www.kaggle.com/steubk/jrsotc-ridgeregression
* https://www.kaggle.com/samarthagarwal23/the-benchmark-0-81-tfidf-ridge

In this version I exploit test labels - realesed after Jigsaw competitions end - to augment trainset

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import rankdata

def ridge_cv (vec, X, y, X_test, folds, stratified ):
    kf = StratifiedKFold(n_splits=FOLDS,shuffle=True,random_state=123)
    val_scores = []
    rmse_scores = []
    X_less_toxics = []
    X_more_toxics = []

    preds = []
    for fold, (train_index,val_index) in enumerate(kf.split(X,stratified)):
        X_train, y_train = X[train_index], y[train_index]
        X_val, y_val = X[val_index], y[val_index]
        model = Ridge()
        model.fit(X_train, y_train)

        rmse_score = mean_squared_error ( model.predict (X_val), y_val, squared = False) 
        rmse_scores.append (rmse_score)

        X_less_toxic = vec.transform(df_val['less_toxic'])
        X_more_toxic = vec.transform(df_val['more_toxic'])

        p1 = model.predict(X_less_toxic)
        p2 = model.predict(X_more_toxic)

        X_less_toxics.append ( p1 )
        X_more_toxics.append ( p2 )

        # Validation Accuracy
        val_acc = (p1< p2).mean()
        val_scores.append(val_acc)

        pred = model.predict (X_test)
        preds.append (pred)

        print(f"FOLD:{fold}, rmse_fold:{rmse_score:.5f}, val_acc:{val_acc:.5f}")

    mean_val_acc = np.mean (val_scores)
    mean_rmse_score = np.mean (rmse_scores)

    p1 = np.mean ( np.vstack(X_less_toxics), axis=0 )
    p2 = np.mean ( np.vstack(X_more_toxics), axis=0 )

    val_acc = (p1< p2).mean()

    print(f"OOF: val_acc:{val_acc:.5f}, mean val_acc:{mean_val_acc:.5f}, mean rmse_score:{mean_rmse_score:.5f}")
    
    preds = np.mean ( np.vstack(preds), axis=0 )
    
    return p1, p2, preds


In [ ]:
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
df_test = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

## Toxic Comment Classification Challenge
* [Jigsaw Toxic Comment Classification Challenge competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data)
* [TensorFlow Datasets link](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes)
* [Kaggle Dataset](https://www.kaggle.com/julian3833/jigsaw-toxic-comment-classification-challenge) thanks to @julian3833 !

This dataset was released for the Jigsaw Toxic Comment Classification Challenge competition on Kaggle

The comments in this dataset come from an archive of Wikipedia talk page comments. 
These have been annotated by Jigsaw human raters for a variety of toxicity subtypes: toxic, severe_toxic, obscene, threat, insult, identity_hate
The toxicity and toxicity subtype labels are binary values (0 or 1) indicating whether the majority of annotators assigned that attribute to the comment text.


**TIP**: After the end of the competition test_labels were released so you can append test data to train to have more data

In [ ]:
features = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]

jc_train_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
print(f"Train: {jc_train_df.shape[0]}")
jc_test_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
temp_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv")
jc_test_df = jc_test_df.merge ( temp_df, on ="id")

#drop test data not used for scoring
jc_test_df = jc_test_df.query ("toxic != -1")
print(f"Test: {jc_test_df.shape[0]}")

jc_df = jc_train_df.append ( jc_test_df ) 
print(f"Train+Test:{jc_df.shape[0]}")

jc_df.head()

In [ ]:
print(f'duplicated by text:{jc_df.duplicated("comment_text").sum()}')

jc_df["toxic_subtype_sum"]=jc_df[features].sum(axis=1)
jc_df["toxic_behaviour"]=jc_df["toxic_subtype_sum"].map(lambda x: x > 0)

tot_toxic_behaviour = jc_df["toxic_behaviour"].sum()
print(f'comments with toxic behaviour:{tot_toxic_behaviour}')


In [ ]:
f,ax=plt.subplots(1,2,figsize=(10,4))
jc_df['toxic_behaviour'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_xlabel('% comments with toxic behaviours')
ax[0].set_ylabel('')

sns.countplot(x='toxic_subtype_sum',data=jc_df.query("toxic_subtype_sum > 0"),ax=ax[1])

plt.suptitle('Toxic Comment Classification Challenge')
plt.show()

df = jc_df.query("toxic_subtype_sum > 0").groupby(features).agg({"id":"count"}).reset_index().sort_values(by="id", ascending=False).head(10)
df = df.rename (columns={"id":"count"})
df["perc"] = df["count"]/tot_toxic_behaviour
df.head(10)

In [ ]:
df = jc_df.query ("toxic == 1 or severe_toxic == 1").groupby(["toxic","severe_toxic"]).agg({"id":"count"}).reset_index()
df = df.rename (columns={"id":"count"})
df["perc"] = df["count"]/tot_toxic_behaviour
df

In [ ]:
df = jc_df.query ("toxic == 0 and toxic_subtype_sum > 0" ).groupby(features).agg({"id":"count"}).reset_index()
df = df.rename (columns={"id":"count"})
df["perc"] = df["count"]/tot_toxic_behaviour
df.sort_values(by="count", ascending=False)

Dataset consist of 223_549 (Train: 159_571, Test: 63_978) not duplicated comments.

10.1% (22_468) comments are labelled with toxic behaviors

32.8% of comments with toxic behaviors are labelled as ("toxic"), while 25.5% as ("toxic","obscene","insult") and 11.6% as ("toxic","obscene") 

8.7% of "toxic" comments are also labelled as "severe_toxic" and "severe_toxic" are always "toxic" comments  

In [ ]:
toxic = 1.0
severe_toxic = 2.0
obscene = 1.0
threat = 1.0
insult = 1.0
identity_hate = 2.0

def create_train (df):
    df['y'] = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].max(axis=1)
    df['y'] = df["y"]+df['severe_toxic']*severe_toxic
    df['y'] = df["y"]+df['obscene']*obscene
    df['y'] = df["y"]+df['threat']*threat
    df['y'] = df["y"]+df['insult']*insult
    df['y'] = df["y"]+df['identity_hate']*identity_hate
    
    
    
    df = df[['comment_text', 'y', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].rename(columns={'comment_text': 'text'})

    #undersample non toxic comments  on Toxic Comment Classification Challenge

    min_len = (df['y'] >= 1).sum()
    df_y0_undersample = df[df['y'] == 0].sample(n=int(min_len*1.5),random_state=201)
    df = pd.concat([df[df['y'] >= 1], df_y0_undersample])
                                                
    return df
 
jc_train_df = create_train (jc_train_df)
jc_test_df = create_train (jc_test_df)


                           
jc_df = jc_train_df.append(jc_test_df)                           

sns.countplot(x='y',data=jc_df)

plt.title('Target distribution for train')
plt.show()



In [ ]:
FOLDS = 5

vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(4, 6) )
X = vec.fit_transform(jc_df['text'])
y = jc_df["y"].values
X_test = vec.transform(df_test['text'])

stratified = np.around ( y )
jc_p1, jc_p2, jc_preds =  ridge_cv (vec, X, y, X_test, FOLDS, stratified )

## Unintended Bias in Toxicity Classification

* [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data)
* [TensorFlow Datasets link](https://www.tensorflow.org/datasets/catalog/civil_comments)
* [Kaggle Dataset](https://www.kaggle.com/julian3833/jigsaw-unintended-bias-in-toxicity-classification) thanks to @julian3833 !


The comments in this dataset come from an archive of the Civil Comments platform, a commenting plugin for independent news sites. These public comments were created from 2015 - 2017 and appeared on approximately 50 English-language news sites across the world. When Civil Comments shut down in 2017, they chose to make the public comments available in a lasting open archive to enable future research.

Jigsaw sponsored this effort and extended annotation of this data by human raters for various toxic conversational attributes.

The original data, published on figshare, includes the public comment text, some associated metadata such as article IDs, timestamps and commenter-generated "civility" labels, but does not include user ids. Jigsaw extended this dataset by adding additional labels for toxicity, identity mentions, as well as covert offensiveness. 

This data set was  released for the Jigsaw Unintended Bias in Toxicity Classification Kaggle challenge. 


Each comment in Train has a toxicity label (target),  this attribute (and all others) are fractional values which represent the fraction of human raters who believed the attribute applied to the given comment. 

The data also has several additional toxicity subtype attributes: severe_toxicity, obscene, threat, insult, identity_attack, sexual_explicit.

*Note that - unlike other Jigsaw dataset - there is not a threat subtype and there is a sexual_explicit subtype*   

Additionally, a subset of comments have been labelled with a variety of identity attributes, representing the identities that are mentioned in the comment (male, female, transgender, ..., black, white, asian, ...)

*Note that the data contains different comments that can have the exact same text. Different comments that have the same text may have been labeled with different targets or subgroups.*


### Labelling Schema

To obtain the toxicity labels, each comment was shown to up to 10 annotators. Annotators were asked to: "Rate the toxicity of this comment"

* Very Toxic (a very hateful, aggressive, or disrespectful comment that is very likely to make you leave a discussion or give up on sharing your perspective)
* Toxic (a rude, disrespectful, or unreasonable comment that is somewhat likely to make you leave a discussion or give up on sharing your perspective)
* Hard to Say
* Not Toxic

These ratings were then aggregated with the target value representing the fraction of annotations that annotations fell within the former two categories.

To collect the identity labels, annotators were asked to indicate all identities that were mentioned in the comment. An example question that was asked as part of this annotation effort was: "What genders are mentioned in the comment?"

* Male
* Female
* Transgender
* Other gender
* No gender mentioned

Again, these were aggregated into fractional values representing the fraction of raters who said the identity was mentioned in the comment.

*Note: Some comments were seen by many more than 10 annotators (up to thousands), due to sampling and strategies used to enforce rater accuracy.*

**TIP**: After the end of the competition test_labels were released so you can append test data to train to have more data

In [ ]:
features = ["toxicity","severe_toxicity","obscene","insult","identity_attack", "sexual_explicit"]
cols = ['id', 'comment_text', 'toxicity', 'severe_toxicity', 'obscene', 'threat','insult', 'identity_attack', 'sexual_explicit', 'toxicity_annotator_count']


juc_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv")
print(f"jigsaw-toxic-comment-classification-challenge shape:{juc_df.shape[0]}")
print(f'duplicated by id:{juc_df.duplicated("id").sum()}, duplicated by text:{juc_df.duplicated("comment_text").sum()}')

juc_df[["id", "comment_text"] + features + ['toxicity_annotator_count']].head()

An importat attribute of this dataset is 'toxicity_annotator_count': it counts the raters of the comment.

**TIP** yuo can use it to remove comments with few raters

In [ ]:
plt.hist (juc_df["toxicity_annotator_count"].clip(0,100), bins=100)
plt.title ("Annotator count distribution")
plt.show()

In [ ]:
juc_df = juc_df.query ("toxicity_annotator_count > 5")
print(f"juc_df:{juc_df.shape}")

juc_df['y'] = juc_df[[ 'severe_toxicity', 'obscene', 'sexual_explicit','identity_attack', 'insult', 'threat']].sum(axis=1)

juc_df['y'] = juc_df.apply(lambda row: row["toxicity"] if row["toxicity"] <= 0.5 else row["y"] , axis=1)
juc_df = juc_df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
min_len = (juc_df['y'] > 0.5).sum()
df_y0_undersample = juc_df[juc_df['y'] <= 0.5].sample(n=int(min_len*1.5),random_state=201)
juc_df = pd.concat([juc_df[juc_df['y'] > 0.5], df_y0_undersample])

plt.hist (juc_df["y"], bins=100)
plt.title('Target distribution for train')
plt.show()

In [ ]:
FOLDS = 5
vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(4, 6) )
X = vec.fit_transform(juc_df['text'])
y = juc_df["y"].values
X_test = vec.transform(df_test['text'])

stratified = (np.around ( y, decimals = 1  )*10).astype(int)
juc_p1, juc_p2, juc_preds =  ridge_cv (vec, X, y, X_test, FOLDS, stratified )

## Ruddit: Norms of Offensiveness for English Reddit Comments

* [Ruddit: Norms of Offensiveness for English Reddit Comments](https://github.com/hadarishav/Ruddit)
* [Kaggle Dataset](https://www.kaggle.com/rajkumarl/ruddit-jigsaw-dataset) thanks to @rajkumarl !
* [Paper](https://aclanthology.org/2021.acl-long.210)


Ruddit is a dataset of English language Reddit comments that has fine-grained, real-valued scores between -1 (maximally supportive) and 1 (maximally offensive).


Offensiveness is evaluated via annotation with Best–Worst Scaling:
"We followed the procedure described in Kiritchenko and Mohammad (2016) to obtain BWS annotations. Annotators were presented with 4 comments (4-tuple) at a time and asked to select the comment that is most offensive (least supportive) and the comment that is least offensive (most supportive). We randomly generated 2N distinct 4-tuples (where N is the number of comments in the dataset), such that each comment was seen in eight different 4-tuples and no two 4-tuples had more than 2 items in common. Kiritchenko and Mohammad (2016) show that in a word-level sentiment task, using just three annotations per 4-tuple produces highly reliable results. However, since we work with long comments and a relatively more difficult task, we got each tuple annotated by 6 annotators. Since each comment is seen in 8 different 4-tuples, we obtain 8 X 6 = 48 udgements per comment."







In [ ]:
rud_df = pd.read_csv("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv")
rud_df.head()

In [ ]:
plt.hist(rud_df['offensiveness_score'], bins=50)
plt.title("Offensiveness distribution")
plt.show()

In [ ]:
print(f"rud_df:{rud_df.shape}")
rud_df['y'] = rud_df['offensiveness_score'].map(lambda x: 0.0 if x <=0 else x)
rud_df = rud_df[['txt', 'y']].rename(columns={'txt': 'text'})
min_len = (rud_df['y'] < 0.5).sum()

plt.hist (rud_df["y"], bins=100)
plt.title('Target distribution for train')
plt.show()

In [ ]:
FOLDS = 5
vec = TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=3, ngram_range=(4, 6) )
X = vec.fit_transform(rud_df['text'])
y = rud_df["y"].values
X_test = vec.transform(df_test['text'])

stratified = (np.around ( y, decimals = 1  )*10).astype(int)
rud_p1, rud_p2, rud_preds =  ridge_cv (vec, X, y, X_test, FOLDS, stratified )

## Ensemble

In [ ]:
jc_max = max(jc_p1.max() , jc_p2.max())
juc_max = max(juc_p1.max() , juc_p2.max())
rud_max = max(rud_p1.max() , rud_p2.max())


p1 = jc_p1/jc_max + juc_p1/juc_max + rud_p1/rud_max
p2 = jc_p2/jc_max + juc_p2/juc_max + rud_p2/rud_max

val_acc = (p1< p2).mean()
print(f"Ensemble: val_acc:{val_acc:.5f}")

# Submission

In [ ]:
score = jc_preds/jc_max + juc_preds/juc_max + rud_preds/rud_max  
## to enforce unique values on score
df_test['score'] = rankdata(score, method='ordinal')

df_test[['comment_id', 'score']].to_csv("submission.csv", index=False)

df_test.head()